In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

pd.set_option('display.max_rows', 200)


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore.csv')
df_reviews=pd.read_csv('https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore_user_reviews.csv')

df.head()



,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


# Operazioni preliminari

Rimozione riga problematica

In [3]:
df[df['Installs']=='Free']


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [4]:
df = df.drop(df[df['Installs']=='Free'].index)

# 3. Transform “Varies with device” into a missing value 

In [5]:
df[df.isin(['Varies with device'])].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varies with device,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df=df.replace('Varies with device',np.nan)

In [7]:
df[df.isin(['Varies with device'])].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#  1. Convert the app sizes to a number

In [8]:
set(df['Size'])

{'1.0M',
 '1.1M',
 '1.2M',
 '1.3M',
 '1.4M',
 '1.5M',
 '1.6M',
 '1.7M',
 '1.8M',
 '1.9M',
 '10.0M',
 '100M',
 '1020k',
 '103k',
 '108k',
 '10M',
 '116k',
 '118k',
 '11M',
 '11k',
 '121k',
 '122k',
 '12M',
 '13M',
 '141k',
 '143k',
 '144k',
 '14M',
 '14k',
 '153k',
 '154k',
 '157k',
 '15M',
 '160k',
 '161k',
 '164k',
 '169k',
 '16M',
 '170k',
 '172k',
 '173k',
 '175k',
 '176k',
 '17M',
 '17k',
 '186k',
 '18M',
 '18k',
 '190k',
 '191k',
 '192k',
 '193k',
 '196k',
 '19M',
 '2.0M',
 '2.1M',
 '2.2M',
 '2.3M',
 '2.4M',
 '2.5M',
 '2.6M',
 '2.7M',
 '2.8M',
 '2.9M',
 '200k',
 '201k',
 '203k',
 '206k',
 '208k',
 '209k',
 '20M',
 '20k',
 '210k',
 '219k',
 '21M',
 '220k',
 '221k',
 '222k',
 '226k',
 '228k',
 '22M',
 '232k',
 '234k',
 '237k',
 '238k',
 '239k',
 '23M',
 '23k',
 '240k',
 '241k',
 '243k',
 '245k',
 '246k',
 '24M',
 '24k',
 '251k',
 '253k',
 '257k',
 '259k',
 '25M',
 '25k',
 '266k',
 '269k',
 '26M',
 '26k',
 '270k',
 '27M',
 '27k',
 '280k',
 '283k',
 '288k',
 '28M',
 '28k',
 '292k',
 '

In [9]:
p=re.compile('(?P<cifre>\d+[.]*\d+)(?P<unità>\w)')

def letter_to_number(letter):
    if letter=='M':
        return 1000000
    if letter=='k':
        return 1000
    return 1

#funzione per la conversione dei valori di Size
def conversion(string):
    if(pd.isnull(string)):
        return string
    else:
        m=p.match(string)
        l=m.group('unità')
        number=letter_to_number(l)
        finalnumber=float(m.group('cifre'))*number
        intnumber=int(finalnumber)
        return intnumber

In [10]:
conversion("1.7M")

1700000

Rimozione missing della colonna Size

In [11]:
full=len(df)
missing=len(df)-len(df.dropna(subset=['Size']))
print(round((missing/full),2))

0.16


In [12]:
df.dropna(subset=['Size'],inplace=True)

Applicazione della funzione di conversione per i valori della colonna Size

In [13]:
df['Size'] = df['Size'].apply(conversion)

In [14]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


# 2. Convert the number of installs to a number 

In [15]:
set(df["Installs"])

{'0+',
 '1+',
 '1,000+',
 '1,000,000+',
 '1,000,000,000+',
 '10+',
 '10,000+',
 '10,000,000+',
 '100+',
 '100,000+',
 '100,000,000+',
 '5+',
 '5,000+',
 '5,000,000+',
 '50+',
 '50,000+',
 '50,000,000+',
 '500+',
 '500,000+',
 '500,000,000+'}

Rimozione dei "+" e delle virgole
Trasformazione dei valori della colonna Installs in numeric

In [16]:
df['Installs']= df['Installs'].str.replace('[+,]','') 
df['Installs']= df['Installs'].apply(lambda x:int(x))
set(df['Installs'])

{0,
 1,
 5,
 10,
 50,
 100,
 500,
 1000,
 5000,
 10000,
 50000,
 100000,
 500000,
 1000000,
 5000000,
 10000000,
 50000000,
 100000000,
 500000000,
 1000000000}

In [17]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


# 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

Funzione per la conversione di Current Version

In [18]:
#regex for Current Version
Cv = re.compile('(?P<version>(\d{1,4}\.){0,4}(\d){1,4})')


def conversion2(string,comp):
    if(pd.isnull(string)):
        return string
    m=comp.search(string)
    if not m:
        return np.nan
    version = m.group('version')
    return version

Funzione per la conversione di Android Version: se ci sono 2 versioni si mantiene la più recente

In [19]:
#regex for Android Version
Av = re.compile('(?P<version>(\d\.){0,2}(\d))\s*\-*\s*(?P<version2>(\d\.){0,2}(\d))*')


def conversion3(string,comp):
    if(pd.isnull(string)):
        return string
    m=comp.search(string)
    if not m:
        return np.nan
    if m.group('version2'):
        version = m.group('version2')
    else:
        version = m.group('version')
    return version

Controllo su un valore di Android version

In [20]:
conversion3('4.0.3 - 7.1.1',Av)

'7.1.1'

In [21]:
df['Current Ver'] = df['Current Ver'].apply(lambda row: conversion2(row,Cv))

In [22]:
df['Android Ver'] = df['Android Ver'].apply(lambda row: conversion3(row,Av))

In [23]:
set(df["Current Ver"])

{nan,
 '3.16.0',
 '0.6.2',
 '3.8',
 '4.7.2',
 '4.8.2.2195',
 '1.0.1802',
 '1.23.1.2',
 '2.2.12',
 '4.11.2',
 '40.10',
 '4.11.3',
 '12.04.18',
 '1.2.12',
 '4.4.4',
 '3.3.8.0308',
 '3.16',
 '9.1.363',
 '4.0.26',
 '1.28.0.5402',
 '1.0.3.8',
 '10.4',
 '1.31.0',
 '1.7.14',
 '4.9.1',
 '1.48.0',
 '2.23',
 '1.9.3097',
 '2.8.9.1004',
 '2.3.13',
 '11.13.18.05.07',
 '1.78',
 '2.3.9',
 '4.11.0',
 '7.3',
 '8.9.9',
 '7.8.8',
 '5.2.2018',
 '5.0.16',
 '2.72',
 '7.1.34.28',
 '1.2016',
 '2.19.2018',
 '5.22.18.0006',
 '1.1.11',
 '1.4.36',
 '1.9.19',
 '1.7.22',
 '1.43',
 '5.68.1',
 '6.2.9',
 '2.106.0',
 '1.01',
 '0.0.52',
 '1.1.07.6',
 '6.4.1',
 '2.17.3',
 '1.441',
 '5.072.1830.2',
 '3.2.8',
 '2.2.0717',
 '1.11.6',
 '1.1.3',
 '3.24.1',
 '1.42.2',
 '2.124.01',
 '2.04',
 '10.0.3086',
 '1.279.13.93',
 '5.3.0.54.7',
 '5.4.7.1',
 '4.54',
 '3.69.0',
 '7.1.4',
 '3.0.1',
 '196',
 '7.7.3',
 '3.5.5',
 '4.2.31',
 '1.5.4.0',
 '5.9.1',
 '2.8.26',
 '5.0.4',
 '5.2.6',
 '7.1.0',
 '2.17',
 '5.11.3',
 '5.1.2',
 '6.7.0',
 '

In [24]:
set(df['Android Ver'])

{'1.0',
 '1.5',
 '1.6',
 '2.0',
 '2.0.1',
 '2.1',
 '2.2',
 '2.3',
 '2.3.3',
 '3.0',
 '3.1',
 '3.2',
 '4.0',
 '4.0.3',
 '4.1',
 '4.2',
 '4.3',
 '4.4',
 '5.0',
 '5.1',
 '6.0',
 '7.0',
 '7.1',
 '7.1.1',
 '8.0',
 nan}

In [25]:
df.head(15)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5600000,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19000000,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29000000,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33000000,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3100000,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3


# 5. Remove the duplicates 

In [26]:
len(df)

9145

In [27]:
df.drop_duplicates(inplace=True)

In [28]:
len(df)

8831

In [29]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4


In [30]:
dup = df.groupby(['App','Category']).size().reset_index()
dup.columns=['App','Category','Size']
dup[dup['Size']>2]

,App,Category,Size
89,8 Ball Pool,GAME,6
601,Amazon Shopping,SHOPPING,3
642,Angry Birds Classic,GAME,5
649,Angry Birds Rio,GAME,4
1377,BeautyPlus - Easy Photo Editor & Selfie Camera,PHOTOGRAPHY,4
1505,Block Puzzle,GAME,3
1622,Bubble Shooter,GAME,5
2136,Camera360: Selfie Photo Editor with Funny Sticker,PHOTOGRAPHY,3
2152,Candy Crush Saga,GAME,4
2153,Candy Crush Soda Saga,GAME,3


Applicando la precedente groupby emergono duplicati: righe con stessa app e stessa categoria

In [31]:
df[df["App"]== "Wordscapes"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1714,Wordscapes,GAME,4.8,230710,87000000,10000000,Free,0,Everyone,Word,"August 2, 2018",1.0.47,4.1
1765,Wordscapes,GAME,4.8,230727,87000000,10000000,Free,0,Everyone,Word,"August 2, 2018",1.0.47,4.1
1889,Wordscapes,GAME,4.8,230849,87000000,10000000,Free,0,Everyone,Word,"August 2, 2018",1.0.47,4.1


In [32]:
df = df.sort_values('Reviews', ascending=False).drop_duplicates(['App', 'Category'],keep='first').sort_index()

Controllo

In [33]:
df[df["App"]== "Wordscapes"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1889,Wordscapes,GAME,4.8,230849,87000000,10000000,Free,0,Everyone,Word,"August 2, 2018",1.0.47,4.1


In [34]:
len(df)

8495

# 6. For each category, compute the number of apps 

In [35]:
df.groupby("Category").size()

Category
ART_AND_DESIGN           62
AUTO_AND_VEHICLES        75
BEAUTY                   47
BOOKS_AND_REFERENCE     194
BUSINESS                376
COMICS                   49
COMMUNICATION           243
DATING                  159
EDUCATION                89
ENTERTAINMENT            64
EVENTS                   57
FAMILY                 1781
FINANCE                 299
FOOD_AND_DRINK           88
GAME                    874
HEALTH_AND_FITNESS      232
HOUSE_AND_HOME           62
LIBRARIES_AND_DEMO       81
LIFESTYLE               334
MAPS_AND_NAVIGATION     107
MEDICAL                 369
NEWS_AND_MAGAZINES      201
PARENTING                54
PERSONALIZATION         351
PHOTOGRAPHY             222
PRODUCTIVITY            291
SHOPPING                168
SOCIAL                  189
SPORTS                  287
TOOLS                   733
TRAVEL_AND_LOCAL        173
VIDEO_PLAYERS           128
WEATHER                  56
dtype: int64

# 7. For each category, compute the average rating

In [36]:
df.groupby("Category")["Rating"].mean()

Category
ART_AND_DESIGN         4.361017
AUTO_AND_VEHICLES      4.147619
BEAUTY                 4.291892
BOOKS_AND_REFERENCE    4.322695
BUSINESS               4.096396
COMICS                 4.168085
COMMUNICATION          4.076720
DATING                 3.963934
EDUCATION              4.373864
ENTERTAINMENT          4.154688
EVENTS                 4.478947
FAMILY                 4.186526
FINANCE                4.104651
FOOD_AND_DRINK         4.108333
GAME                   4.236014
HEALTH_AND_FITNESS     4.191099
HOUSE_AND_HOME         4.128000
LIBRARIES_AND_DEMO     4.203279
LIFESTYLE              4.089963
MAPS_AND_NAVIGATION    4.008511
MEDICAL                4.165169
NEWS_AND_MAGAZINES     4.143506
PARENTING              4.347727
PERSONALIZATION        4.325362
PHOTOGRAPHY            4.114216
PRODUCTIVITY           4.132735
SHOPPING               4.213699
SOCIAL                 4.257692
SPORTS                 4.204036
TOOLS                  4.007484
TRAVEL_AND_LOCAL       4.011348

# 8. Create two dataframes: one for the genres and one bridging apps and genres.
## So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

Creazione del dataset Genres, con valori della colonna Genres  successivamente splittati in due differenti colonne. 

Si considera anche la colonna Rating, utile per analisi successive

In [37]:
genres = df[['App','Genres','Rating']]
genres.head()

,App,Genres,Rating
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design,4.1
1,Coloring book moana,Art & Design;Pretend Play,3.9
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design,4.7
3,Sketch - Draw & Paint,Art & Design,4.5
4,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity,4.3


In [38]:
genres1 = genres['Genres'].str.split(';',expand=True)[0]
genres2 = genres['Genres'].str.split(';',expand=True)[1]

In [39]:
genres_final=pd.concat([genres,genres1,genres2],axis=1)
genres_final.columns=['App','Genres','Rating','Genre1','Genre2']
genres_final.drop('Genres', axis=1,inplace=True)
genres_final.head()

,App,Rating,Genre1,Genre2
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,Art & Design,None
1,Coloring book moana,3.9,Art & Design,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,Art & Design,None
3,Sketch - Draw & Paint,4.5,Art & Design,None
4,Pixel Draw - Number Art Coloring Book,4.3,Art & Design,Creativity


Bridging Dataframe 
Selezione di tutte le app con Genre2 (il secondo genere di appartenenza) non nullo

In [40]:
genre_2 = genres_final[['App','Rating','Genre2']]
genre_2 = genre_2[genre_2['Genre2'].notnull()]
genre_2.head()

,App,Rating,Genre2
1,Coloring book moana,3.9,Pretend Play
4,Pixel Draw - Number Art Coloring Book,4.3,Creativity
9,Kids Paint Free - Drawing Fun,4.7,Creativity
23,Mcqueen Coloring pages,NaN,Action & Adventure
26,Colorfit - Drawing & Coloring,4.7,Creativity


In [43]:
len(genre_2)

370

Creazione indice, dal 10840 in poi, per 370 osservazioni di genre_2

In [54]:
idx=list(range(10840,10840+370))
genre_2.index=idx

Creazione di  un nuovo dataset escludendo la colonna Genre2 e rinominazione  delle colonne dei dataset per applicare il concat

In [55]:

genre_1=genres_final[['App','Rating','Genre1']]

genre_1.columns = ['App','Rating','Genre']
genre_2.columns = ['App','Rating','Genre']
df_bridging=pd.concat([genre_1,genre_2])

In [58]:
len(df_bridging)

8865

Controllo sull'App "Coloring book moana"

In [59]:

df_bridging[df_bridging["App"]=="Coloring book moana"]

,App,Rating,Genre
1,Coloring book moana,3.9,Art & Design
2033,Coloring book moana,3.9,Art & Design
10840,Coloring book moana,3.9,Pretend Play
10893,Coloring book moana,3.9,Pretend Play


Creazione dataset contentente solo i generi delle App

In [61]:
df_genres=genres_final.drop(['App','Rating'], axis=1)
df_genres.head()

,Genre1,Genre2
0,Art & Design,None
1,Art & Design,Pretend Play
2,Art & Design,None
3,Art & Design,None
4,Art & Design,Creativity


# 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [62]:
df_bool = df["Genres"].str.get_dummies(";")
df_bool.head()

,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Brain Games,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
df = pd.concat([df,df_bool],axis=1)

In [64]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone,Art & Design;Pretend Play,...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen,Art & Design,...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone,Art & Design;Creativity,...,0,0,0,0,0,0,0,0,0,0


Controllo sulla riga 1 che ha 2 generi: "Art & Design" e "Pretend Play"

In [65]:
df.loc[1]

App                              Coloring book moana
Category                              ART_AND_DESIGN
Rating                                           3.9
Reviews                                          967
Size                                        14000000
Installs                                      500000
Type                                            Free
Price                                              0
Content Rating                              Everyone
Genres                     Art & Design;Pretend Play
Last Updated                        January 15, 2018
Current Ver                                    2.0.0
Android Ver                                    4.0.3
Action                                             0
Action & Adventure                                 0
Adventure                                          0
Arcade                                             0
Art & Design                                       1
Auto & Vehicles                               

# 10. For each genre, compute the average rating. What is the genre with highest average? 

In [66]:
df_bridging.groupby('Genre')['Rating'].mean()

Genre
Action                     4.237716
Action & Adventure         4.285393
Adventure                  4.193333
Arcade                     4.276163
Art & Design               4.359016
Auto & Vehicles            4.147619
Beauty                     4.291892
Board                      4.294444
Books & Reference          4.320979
Brain Games                4.359016
Business                   4.096396
Card                       4.051220
Casino                     4.268750
Casual                     4.143000
Comics                     4.168085
Communication              4.076720
Creativity                 4.335484
Dating                     3.963934
Education                  4.295413
Educational                4.091954
Entertainment              4.093126
Events                     4.478947
Finance                    4.104651
Food & Drink               4.108333
Health & Fitness           4.192228
House & Home               4.128000
Libraries & Demo           4.203279
Lifestyle             

In [67]:
df_bridging.groupby('Genre')['Rating'].mean().idxmax()

'Events'

# 11. For each app, compute the approximate income, obtain as a product of number of installs and price

In [68]:
set(df["Price"])

{'$0.99',
 '$1.00',
 '$1.04',
 '$1.20',
 '$1.26',
 '$1.29',
 '$1.49',
 '$1.50',
 '$1.59',
 '$1.61',
 '$1.70',
 '$1.75',
 '$1.76',
 '$1.96',
 '$1.97',
 '$1.99',
 '$10.00',
 '$10.99',
 '$109.99',
 '$11.99',
 '$12.99',
 '$13.99',
 '$14.00',
 '$14.99',
 '$15.46',
 '$15.99',
 '$154.99',
 '$16.99',
 '$17.99',
 '$18.99',
 '$19.40',
 '$19.90',
 '$19.99',
 '$2.00',
 '$2.49',
 '$2.56',
 '$2.59',
 '$2.60',
 '$2.90',
 '$2.99',
 '$200.00',
 '$24.99',
 '$25.99',
 '$28.99',
 '$29.99',
 '$299.99',
 '$3.02',
 '$3.04',
 '$3.08',
 '$3.28',
 '$3.49',
 '$3.61',
 '$3.88',
 '$3.99',
 '$30.99',
 '$33.99',
 '$37.99',
 '$379.99',
 '$389.99',
 '$39.99',
 '$394.99',
 '$399.99',
 '$4.29',
 '$4.49',
 '$4.60',
 '$4.77',
 '$4.80',
 '$4.84',
 '$4.85',
 '$4.99',
 '$400.00',
 '$46.99',
 '$5.00',
 '$5.49',
 '$5.99',
 '$6.49',
 '$6.99',
 '$7.49',
 '$7.99',
 '$74.99',
 '$79.99',
 '$8.49',
 '$8.99',
 '$89.99',
 '$9.00',
 '$9.99',
 '0'}

Rimozione del $ dalla stringa e conseguente trasformazione in float

In [69]:
df['Price']=df['Price'].str.replace('[$]','') 
df['Price']=df['Price'].apply(lambda x:float(x))
set(df['Price'])

{0.0,
 0.99,
 1.0,
 1.04,
 1.2,
 1.26,
 1.29,
 1.49,
 1.5,
 1.59,
 1.61,
 1.7,
 1.75,
 1.76,
 1.96,
 1.97,
 1.99,
 2.0,
 2.49,
 2.56,
 2.59,
 2.6,
 2.9,
 2.99,
 3.02,
 3.04,
 3.08,
 3.28,
 3.49,
 3.61,
 3.88,
 3.99,
 4.29,
 4.49,
 4.6,
 4.77,
 4.8,
 4.84,
 4.85,
 4.99,
 5.0,
 5.49,
 5.99,
 6.49,
 6.99,
 7.49,
 7.99,
 8.49,
 8.99,
 9.0,
 9.99,
 10.0,
 10.99,
 11.99,
 12.99,
 13.99,
 14.0,
 14.99,
 15.46,
 15.99,
 16.99,
 17.99,
 18.99,
 19.4,
 19.9,
 19.99,
 24.99,
 25.99,
 28.99,
 29.99,
 30.99,
 33.99,
 37.99,
 39.99,
 46.99,
 74.99,
 79.99,
 89.99,
 109.99,
 154.99,
 200.0,
 299.99,
 379.99,
 389.99,
 394.99,
 399.99,
 400.0}

In [70]:
df['Approximate Income']= round(df['Installs']*df['Price'],2)
set(df['Approximate Income'])

{0.0,
 0.99,
 1.49,
 1.99,
 2.49,
 2.99,
 3.99,
 4.95,
 5.99,
 7.45,
 9.9,
 9.95,
 14.9,
 19.6,
 19.9,
 20.0,
 24.9,
 28.99,
 29.9,
 29.95,
 30.4,
 34.9,
 36.1,
 44.95,
 48.0,
 49.5,
 49.9,
 52.0,
 54.9,
 59.9,
 64.9,
 74.5,
 87.5,
 88.0,
 99.0,
 99.5,
 100.0,
 109.99,
 126.0,
 130.0,
 149.0,
 149.5,
 149.9,
 199.0,
 199.5,
 199.9,
 199.95,
 242.5,
 249.0,
 249.5,
 259.9,
 299.0,
 299.5,
 299.9,
 302.0,
 349.0,
 388.0,
 399.0,
 449.0,
 449.5,
 469.9,
 495.0,
 499.0,
 499.5,
 500.0,
 599.0,
 649.0,
 699.0,
 745.0,
 749.5,
 799.0,
 899.9,
 990.0,
 995.0,
 999.0,
 1000.0,
 1245.0,
 1290.0,
 1299.0,
 1400.0,
 1490.0,
 1495.0,
 1500.0,
 1610.0,
 1700.0,
 1745.0,
 1990.0,
 1995.0,
 2245.0,
 2490.0,
 2495.0,
 2499.0,
 2560.0,
 2990.0,
 2995.0,
 3080.0,
 3490.0,
 3990.0,
 3999.0,
 4290.0,
 4490.0,
 4495.0,
 4840.0,
 4950.0,
 4990.0,
 5000.0,
 5990.0,
 6990.0,
 7450.0,
 7490.0,
 7990.0,
 7995.0,
 8490.0,
 8995.0,
 9900.0,
 9950.0,
 9990.0,
 10000.0,
 10990.0,
 12000.0,
 12450.0,
 14950.0,
 1499

In [76]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word,Approximate Income
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0.0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,0,0,0,0,0,0,0,0,0,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0.0,Everyone,Art & Design,...,0,0,0,0,0,0,0,0,0,0.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0.0,Teen,Art & Design,...,0,0,0,0,0,0,0,0,0,0.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0.0,Everyone,Art & Design;Creativity,...,0,0,0,0,0,0,0,0,0,0.0


# 12. For each app, compute its minimum and maximum Sentiment_polarity 

In [71]:
reviews=pd.read_csv('https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore_user_reviews.csv')
reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


Valori massimi

In [72]:
reviews.groupby('App')['Sentiment_Polarity'].max().sort_values(ascending=False)

App
10 Best Foods for You                                 1.0
FBReader: Favorite Book Reader                        1.0
Eve Period Tracker - Love, Sex & Relationships App    1.0
BigOven Recipes, Meal Planner, Grocery List & More    1.0
Evie Launcher                                         1.0
ExDialer - Dialer & Contacts                          1.0
Expedia Hotels, Flights & Car Rental Travel Deals     1.0
Bible                                                 1.0
BeyondMenu Food Delivery                              1.0
Betterment                                            1.0
Extreme Racing 2 - Real driving RC cars game!         1.0
EyeEm - Camera & Photo Filter                         1.0
EzCalculator                                          1.0
BestCam Selfie-selfie, beauty camera, photo editor    1.0
Blood Pressure Log - MyDiary                          1.0
Best Wallpapers QHD                                   1.0
FOSSIL Q: DESIGN YOUR DIAL                            1.0
FOX       

Valori minimi

In [73]:
reviews.groupby('App')['Sentiment_Polarity'].min().sort_values()

App
BestCam Selfie-selfie, beauty camera, photo editor   -1.0
BuzzFeed: News, Tasty, Quizzes                       -1.0
Extreme Match                                        -1.0
Expedia Hotels, Flights & Car Rental Travel Deals    -1.0
Eat Fit - Diet and Health Free                       -1.0
Anthem Anywhere                                      -1.0
Anthem BC Anywhere                                   -1.0
Easy Origami Ideas                                   -1.0
Easy Makeup Tutorials                                -1.0
Easy Installer - Apps On SD                          -1.0
Apartment, Home Rental Search: Realtor.com Rentals   -1.0
DuraSpeed                                            -1.0
Dungeon Hunter Champions: Epic Online Action RPG     -1.0
DroidAdmin for Android - Advice                      -1.0
BÁO MỚI - Đọc Báo, Tin Tức 24h                       -1.0
CAIXA                                                -1.0
Dr. Panda Restaurant 3                               -1.0
Doodle Jum